In [1]:
from mxnet import autograd, nd
from mxnet.gluon import nn

In [2]:
#define cross correlation
def corr2d(X, K):
    h, w = K.shape
    #print('X:')
    #print(X)
    #print('K:')
    #print(K)
    #print('K.shape:')
    #print(K.shape)
    Y = nd.zeros((X.shape[0] -h +1, X.shape[1] -w +1))
    #print(Y)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            #print('i = %d j = %d h = %d w = %d' %(i, j, h, w))
            Y[i, j] = (X[i:i + h, j:j +w] * K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])

corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

In [4]:
#define 2d convolution layer
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias =self.params.get('bias', shape=(1,))
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

In [5]:
#detect edge
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
K = nd.array([[1, -1]])

In [7]:
Y = corr2d(X, K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [13]:
#leaning kernel array from data
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

#batch_size, channel, high, width
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat -Y)**2
    l.backward()
    
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0 :
        print('batch %d, loss %.3f' % (i+1, l.sum().asscalar()))

batch 2, loss 5.132
batch 4, loss 0.871
batch 6, loss 0.150
batch 8, loss 0.027
batch 10, loss 0.005


In [15]:
conv2d.weight.data().reshape((1, 2))


[[ 0.9917276  -0.98480207]]
<NDArray 1x2 @cpu(0)>